In [24]:
import pandas as pd
import sqlite3

from src.input.caiso_connector import (
    download_csv_file,
    unzip_csv,
    delete_data_files,
    find_csv_files,
    ZIP_DIRECTORY,
    target_url
)

# Connect to database 
conn = sqlite3.connect("src/output/lmp.db")

# Download file

In [25]:
target_url

'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20190201T00:00-0000&enddatetime=20190206T00:00-0000&version=1&market_run_id=DAM&grp_type=ALL_APNODES&resultformat=6'

In [27]:
# example target urls 
target_1 = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20190207T00:00-0000&enddatetime=20190208T00:00-0000&version=1&market_run_id=DAM&grp_type=ALL_APNODES&resultformat=6'
target_2 = 'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20190208T00:00-0000&enddatetime=20190209T00:00-0000&version=1&market_run_id=DAM&grp_type=ALL_APNODES&resultformat=6'
targets = [target_1, target_2]

In [28]:
for target in targets:
    download_csv_file(target, ZIP_DIRECTORY)
    unzip_csv(ZIP_DIRECTORY)

# Load dataframe 

In [31]:
lmp_columns = ['INTERVALSTARTTIME_GMT', 'NODE','LMP_TYPE','MW']
for file in find_csv_files(ZIP_DIRECTORY):
    df = (pd.read_csv(file, usecols=lmp_columns)
          .rename(columns = {'INTERVALSTARTTIME_GMT':'time','NODE':'node','MW':'mw'}))
    df.time = pd.to_datetime(df.time)
    df = df[df["LMP_TYPE"]=="LMP"].drop(columns=['LMP_TYPE'])
    df.to_sql("lmp", conn, if_exists="append", index=False)
    
delete_data_files(ZIP_DIRECTORY)

In [ ]:
print(len(df['NODE'].unique()))

In [ ]:
for item in df['NODE'].unique():
    print(item)

In [ ]:
df[df['NODE']=="AFPR_1_TOT_GEN-APND"][['MW']]

In [16]:
df.to_sql("lmp", conn, if_exists="append", index=False)

In [18]:
pd.read_sql_query("select * from lmp;", conn)

,time,node,mw
0,2019-02-01 01:00:00+00:00,AFPR_1_TOT_GEN-APND,59.28223
1,2019-02-01 00:00:00+00:00,AFPR_1_TOT_GEN-APND,43.21976
2,2019-02-01 02:00:00+00:00,AFPR_1_TOT_GEN-APND,60.94596
3,2019-02-01 06:00:00+00:00,AFPR_1_TOT_GEN-APND,41.12965
4,2019-02-01 07:00:00+00:00,AFPR_1_TOT_GEN-APND,38.83438
...,...,...,...
190419,2019-02-05 17:00:00+00:00,YALE_7_UNITS-APND,50.69935
190420,2019-02-05 12:00:00+00:00,YALE_7_UNITS-APND,50.14012
190421,2019-02-05 19:00:00+00:00,YALE_7_UNITS-APND,42.37442
190422,2019-02-05 08:00:00+00:00,YALE_7_UNITS-APND,52.92776


In [33]:
pd.read_sql_query(
    """select * from lmp
    WHERE time > "2019-02-05";
    """,
    conn)

,time,node,mw
0,2019-02-05 04:00:00+00:00,AFPR_1_TOT_GEN-APND,56.62196
1,2019-02-05 03:00:00+00:00,AFPR_1_TOT_GEN-APND,59.95031
2,2019-02-05 06:00:00+00:00,AFPR_1_TOT_GEN-APND,40.71352
3,2019-02-05 07:00:00+00:00,AFPR_1_TOT_GEN-APND,38.52087
4,2019-02-05 01:00:00+00:00,AFPR_1_TOT_GEN-APND,57.86465
...,...,...,...
152347,2019-02-05 17:00:00+00:00,YALE_7_UNITS-APND,50.69935
152348,2019-02-05 12:00:00+00:00,YALE_7_UNITS-APND,50.14012
152349,2019-02-05 19:00:00+00:00,YALE_7_UNITS-APND,42.37442
152350,2019-02-05 08:00:00+00:00,YALE_7_UNITS-APND,52.92776


In [32]:
pd.read_sql_query(
    """select COUNT(*) from lmp;
    """,
    conn)

,COUNT(*)
0,457024
